**Author: Antonio Moreno Martin**

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly.express as px
import seaborn as sns
import glob
import xgboost as xgb
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMAResults
from statsmodels.graphics.tsaplots import plot_predict
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def create_features(df):
    '''
    Crearemos features de time series basados en el index para estudiar luego su comportamiento
    '''
    
    df = df.set_index('Fecha')
    df['month'] = df.index.month
    df['year'] = df.index.year
    
    return df

def plot_prediction(df_total, df):
    fig = make_subplots(rows=1, cols=1)


    train =df_total.loc[:(len(df)-1)]
    prediction = df_total.loc[(len(df)-1):]
    prediction

    fig.add_trace(
        go.Line(name = "Precio Real", x = train.Fecha, y = train.Precio),
        row=
        1, col=1
    )
    fig.add_trace(        
        go.Line(name = "Precio Predicho", x = prediction.Fecha, y = prediction.Precio),
        row=
        1, col=1
    )
    fig
    fig.show()

### Read and gather downloaded data

In [ ]:
# Leer y gather data
input = "/Users/amm/Documents/Github/Data/Gasolina/source/Madrid_alcampo_gasolina_98_E5/"
files = glob.glob(input + '*.xls')

df = pd.concat([pd.read_excel(file) for file in files], ignore_index=True)

df['Fecha'] = pd.to_datetime(df['Fecha'])
df.sort_values(by='Fecha', inplace = True)

### <mark> Define dates to train and test </mark>

In [ ]:
time_train = '2023-04-01' # This is the last date of the train
time_test = '2023-12-01' # This is the last date of the test
time_to_predict = (datetime.strptime(time_train, '%Y-%m-%d')+ relativedelta(months=1)).strftime('%Y-%m-%d') # we add one month to start the prediction and convert back to string
df_train = df[df['Fecha']<=time_train]
df_test = df[df['Fecha']>time_train]
df_train

In [ ]:

fig = px.line(df, x = 'Fecha', y = 'Precio', title = 'Gasolina 98 E5 Madrid 2020 - 2023')
fig.show()

In [ ]:

df_study = create_features(df)
fig = px.box(df_study, x= "month", y="Precio")
fig.show()

In [ ]:

# Determinar si es un random walk o no
# Vamos a determinar si es un random walk o no. Recordar que es un proceso donde hay mismas posibilidades tanto de ir hacia arriba o hacia abajo por un número aleatorio.
# Step 1: Ver si existe una tendencia. En este caso, parece que puede haberlo ya que año a año ha ido incrementando.
## 1.a Vamos a descomponerlo en tendencia, temporalidad y residuos

In [ ]:
advanced_decomposition = STL(df['Precio'], period = 12).fit()

fig = make_subplots(rows=4, cols=1, subplot_titles=("Observed", "Trend", "Seasonal", "Residuals"))

fig.add_trace(
    go.Line(name = "Valores reales", x = df.Fecha, y = advanced_decomposition.observed),
    row=1, col=1
)
fig.add_trace(
    go.Line(name = "Tendencia", x = df.Fecha, y = advanced_decomposition.trend),
    row=2, col=1
)
fig.add_trace(
    go.Line(name = "Componente Estacional", x = df.Fecha, y = advanced_decomposition.seasonal),
    row=3, col=1
)
fig.add_trace(
    go.Line(name = "Residuos", x = df.Fecha, y = advanced_decomposition.resid),
    row=4, col=1
)

Efectivamente, vemos una tendencia en la segunda gráfica. Vamos a salir de dudas con test ADF para ver el ACF(Autocorrelation function):

In [ ]:
ADF_result = adfuller(df.Precio)

print(f'ADF Statistic: {ADF_result[0]}')
print(f'p-value: {ADF_result[1]}')

plot_acf(df.Precio, lags=12) # Vemos como hay una relación linela en las muestras y por tanto, es no estacionario. Dentro del confidence interval se considera que es como tener 

Como el p valores mayor que 0.05, no podemos rechazar la null hyphotesis y por tanto es no estacionacionaria. Por ende, tennemos que volver a diferenciar por primera vez.

## Plot ACF

### Aplicar Dickey-Fuller test (ADF)  para saber si es temporal o no

In [ ]:

diff_gasolina = np.diff(df['Precio'], n = 1)
print(f'ADF Statistic: {diff_gasolina[0]}')
print(f'p-value: {diff_gasolina[1]}')
plot_acf(diff_gasolina, lags = 12) 



In [ ]:
ADF_result = adfuller(diff_gasolina)
print(f'ADF Statistic: {ADF_result[0]}')
print(f'p-value: {ADF_result[1]}')

**p valor menor que 0,05 y un ADF negativo no muy grande => por tanto, podemos rechazar la hipotesis nula y decir que es <mark>estacional</mark>**

Vemos como tiene un comportamiento <mark> SINUSOIDAL pattern</mark> 

# Moving Average (MA)

<mark>Uno de los motivos por los que hariamos MA, sería al dibujar el PACF (funcion parcial de autocorrelacion), donde tendríamos que ver un decay LINEAL. Recordamos que el MA era decay exponencial</mark>

In [ ]:
df_aux = df.copy()
df_aux = df_aux.set_index('Fecha')


In [ ]:
df_aux = df.copy()
df_aux = df_aux.set_index('Fecha')

train = df_aux.loc[df_aux.index <= time_train]
test = df_aux.loc[df_aux.index > time_train]
train

In [ ]:
diff_train = np.diff(train['Precio'], n = 1)
indices_diff = train[1:].index.tolist()
df_diff = pd.DataFrame({'diff_gasolina': diff_train})
df_diff.index = indices_diff
df_diff


In [ ]:
mod = ARIMA(df_diff, order=(0,0,1))
res = mod.fit()

predict_ma = res.get_prediction(start = time_to_predict, end = time_test)
prediction_ma = pd.DataFrame( columns=['predicted_MA'])

### <mark>`len(df) - 1`</mark> se pone ya que cuando diferenciamos, el primer elemento se pierde, y por tanto, no podemos considerar el primer elemento que se encuentra en el lag 0

In [ ]:
df_pred = pd.DataFrame({'Fecha':predict_ma.predicted_mean.index, 'Precio':predict_ma.predicted_mean.values})
df_ma = pd.concat([df_train, df_pred])
df_ma = df_ma.reset_index(drop=True)

df_ma['Precio'].loc[(len(df_train)-1):,] = df_ma['Precio'][(len(df_train)-1):].cumsum()
df_ma


## PLOT MA PREDICTION

In [ ]:
plot_prediction(df_ma, df_train)

### Calculate the error with the predictions made

## MSE

- El <mark>len(df)</mark> del modelo con el que hemos construido el MA es en este momento <mark>136</mark>, por ello, para calcular el error compararemos hasta ese número

In [ ]:
mse_ma = mean_squared_error(df_test['Precio'].loc[list(df_test.index)], df_ma['Precio'].loc[list(df_test.index)])
mse_ma

# AUTOREGRESSIVE MODEL (AR)

<mark>Al igual que para MA, es necesario dibujar su ACF, pero esta ve en lugar de decaer linealmente, deberíamos cerciorarnos de que decae exponencialmente<br>
Por otra parte, a DIFERENCIA de MA, para conocer su grado MA(p), NO nos basta  con el ACF, en este caso debemos dibujar el PACF y ver donde deja de haber coeficientes no significativos</mark>

## Plot PACF

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(diff_gasolina, lags=20);

<mark>Vemos, que a partir del lag 2, los coeficientes dejan de ser significativos, por ende, estamos frente a un AR(2)
<br>
Además, cabe destacar que tiene al igual que el ACF, un comportamiento sinusoidal, por ende, podemos aplicar un modelo ARMA</mark>

In [ ]:
mod = ARIMA(df_diff, order=(2,0,0)) # AR(2)
res = mod.fit()

predict_ar = res.get_prediction(start = time_to_predict, end = time_test)
prediction_ar = pd.DataFrame( columns=['predicted_AR'])

In [ ]:
print(res.summary())

- const = 0 y ar.L1 0.3 es igual a phi=0.3 en la ecuación matemática

In [ ]:
print(res.params)

In [ ]:
df_pred =pd.DataFrame({'Fecha':predict_ar.predicted_mean.index, 'Precio':predict_ar.predicted_mean.values})
df_ar = pd.concat([df_train, df_pred])
df_ar = df_ar.reset_index(drop=True)

df_ar['Precio'].loc[(len(df_train)-1):,] = df_ar['Precio'][(len(df_train)-1):].cumsum()
df_ar


## PLOT AR PREDICTION

In [ ]:
plot_prediction(df_ar, df_train)

## MSE

In [ ]:
mse_ar = mean_squared_error(df_test['Precio'].loc[list(df_test.index)], df_ar['Precio'].loc[list(df_test.index)])
mse_ar

# AUTOREGRESSIVE MOVING AVERAGE (ARMA)

<mark>Para identificar si es un ARMA, tenemos que visualizar unos patrones sinusoidales o un patrón decaying TANTO en el ACF Y PACF por el cual no haya un claro lag donde se vuelva abruptamente no significante.</mark>

Referencia:
<br>
<br>
https://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_0.html
<br>
<br>
<mark> En funcion ARIMA(p, d, q) los parámetros que podemos rellenar para usar un ARMA son p y q, por ende, la d tiene que ser igual a cero cuando construyamos un ARMA model </mark>


En nuestro caso, podemos ver como en ambos casos es sinusoidal, así que procederemos a aplicar un ARMA

## AIC to create the model

In [ ]:
def create_opt_ARIMA(df_to_predict, grades, d):
    '''
    Función para fittear distintos ARMAS o ARIMAS con todas las combinaciones de grades que le indicaremos, eligiendo el modelo más óptimo (viendo su AIC)
    Si queremos crear un ARMA, habrá que poner la 'd' a cero.
    
    df_to_predict: pandas dataframe
        Es el df ya tratado previamente para poder aplicarle el modelo ARMA y predecir
    
    grades: int MAYOR QUE CERO
        Creará una lista de 0 hasta grades
        
    '''
    grades = list(range(0,grades + 1))
    arma_mods = [ARIMA(df_to_predict, order=(p, d, q)).fit() for q in grades for p in grades]
    aic = [arma.aic for arma in arma_mods]
    ar = [list(arma.model_orders.values())[2]for arma in arma_mods] # Elegimos el valor 2, ya que en model_orders, es donde se encuentra el parametro p correspondiente al ar
    ma = [list(arma.model_orders.values())[3]for arma in arma_mods]
    
    df_armas = pd.DataFrame(data = zip(ar, ma, aic), columns = ["p", "q", "AIC"])
    df_armas = df_armas.sort_values("AIC") # Ordenamos de MENOR a MAYOR, ya que el menor AIC  
    return df_armas, arma_mods

<mark>The lower the value for AIC, the better the fit of the model. The absolute value of the AIC value is not important. It can be positive or negative.</mark>
<br>
<br>
**Reference**: https://www.statology.org/negative-aic/

In [ ]:
df_armas, arma_mods = create_opt_ARIMA(df_diff, 3, 0)
df_armas

In [ ]:
# Seleccionamos el primer modelo al ser el que tiene menor AIC, por eso elegimos el [0]
arma_model = arma_mods[df_armas.index.tolist()[0]]
arma_model.summary()

## Model's residuals

Hasta este punto, viendo el AIC, nos damos cuenta que el mejor modelo es el ARMA(2,2). Ahora, debemos <mark>analizar los residuos</mark>, que no dejan de ser la diferencia entre el valor real
<br> y el predicho para ver que dicha diferencia no es random. Eso se mira a través del <mark>QQplot</mark> y corriendo el <mark>Ljung-Boxtest</mark>

In [ ]:
residuals = arma_model.resid
residuals

### QQ-PLOT

Es una herramienta gráfica para verificar la hipotesis de que nuestro model's residuals tiene distribución normal.
<br> Si el scatterplot se superpone a la lineal, diremos que tiene buen fit 

In [ ]:
arma_model.plot_diagnostics(figsize=(10, 8));

In [ ]:
arma_predict = arma_model.get_prediction(start = time_to_predict, end = time_test)

In [ ]:
df_pred = pd.DataFrame({'Fecha':arma_predict.predicted_mean.index, 'Precio':arma_predict.predicted_mean.values})
df_arma = pd.concat([df_train, df_pred])
df_arma = df_arma.reset_index(drop=True)

df_arma['Precio'].loc[(len(df_train)-1):,] = df_arma['Precio'][(len(df_train)-1):].cumsum()
df_arma


## PLOT ARMA PREDICTION

In [ ]:
plot_prediction(df_arma, df_train)

## MSE

In [ ]:
mse_arma = mean_squared_error(df_test['Precio'].loc[list(df_test.index)], df_arma['Precio'].loc[list(df_test.index)])
mse_arma

# AUTOREGRESSIVE INTEGRATED MOVING AVERAGE MODEL (ARIMA)

Añadiremos una componente a ARMA llamada el integration order denotado con la letra d. Con esta componente evitaremos tener en cuenta la no estacionalidad de las time series y evitar los pasos de transformación y diferenciación. Esta componente es simplemente la inversa de la diferenciación. <mark>Por ende, el orden de integración es igual al número de veces que una serie ha sido diferenciada hasta que se ha vuelto estacional. Por ejemplo, si diferenciamos una serie dos veces y se vuelve estacional d=2</mark>
<br>
<br>
<mark>Resumiendo, el modelo ARIMA es como un modelo ARMA que puede ser aplicado en una serie no temporal sin necesidad de hacer todas las transformaciones que hacíamos, simplemente, ahora, tenemos que encontrar el orden de integración necesario para hacer la serie estacional </mark>

### Primer paso

Tenemos que conocer el orden de diferenciación hasta que nuestra serie sea estacional. Por ello, iríamos aplicando el test de Dickey-Fuller a cada serie hasta que podamos rechazar la hipotesis nula y decir que es estacional (siempre que la serie original no fuese estacional).
<br>
<br>
**Como dicho trabajo ya lo hemos hecho previamente, sabemos que el orden de diferenciación es en este caso 1 => d = 1**

In [ ]:
def create_opt_ARIMA(df_to_predict, grades, d):
    '''
    Función para fittear distintos ARMAS o ARIMAS con todas las combinaciones de grades que le indicaremos, eligiendo el modelo más óptimo (viendo su AIC)
    Si queremos crear un ARMA, habrá que poner la 'd' a cero.
    
    df_to_predict: pandas dataframe
        Es el df ya tratado previamente para poder aplicarle el modelo ARMA y predecir
    
    grades: int MAYOR QUE CERO
        Creará una lista de 0 hasta grades
        
    d: orden de diferenciacion, cuantas veces hay que diferenciar la serie para hacerla estacional
        
    '''
    grades = list(range(0,grades + 1))
    arma_mods = [ARIMA(df_to_predict, order=(p, d, q)).fit() for q in grades for p in grades]
    aic = [arma.aic for arma in arma_mods]
    ar = [list(arma.model_orders.values())[2]for arma in arma_mods] # Elegimos el valor 2, ya que en model_orders, es donde se encuentra el parametro p correspondiente al ar
    ma = [list(arma.model_orders.values())[3]for arma in arma_mods]
    
    df_armas = pd.DataFrame(data = zip(ar, ma, aic), columns = ["p", "q", "AIC"])
    df_armas = df_armas.sort_values("AIC") # Ordenamos de MENOR a MAYOR, ya que el menor AIC  
    return df_armas, arma_mods

In [ ]:
df_arimas, arima_mods = create_opt_ARIMA(df_diff, 3, d = 1)
df_arimas

In [ ]:
# Seleccionamos el primer modelo al ser el que tiene menor AIC, por eso elegimos el [0]
arima_model = arima_mods[df_arimas.index.tolist()[0]]
arima_model.summary()

In [ ]:
arma_model.plot_diagnostics(figsize=(10, 8));

In [ ]:
arima_predict = arima_model.get_prediction(start = time_to_predict, end = time_test)
prediction_arima = pd.DataFrame( columns=['predicted_ARIMA'])

In [ ]:
df_pred = pd.DataFrame({'Fecha':arima_predict.predicted_mean.index, 'Precio':arima_predict.predicted_mean.values})
df_arima = pd.concat([df_train, df_pred])
df_arima = df_arima.reset_index(drop=True)

df_arima['Precio'].loc[(len(df_train)-1):,] = df_arima['Precio'][(len(df_train)-1):].cumsum()
df_arima


## PLOT ARIMA PREDICTION

In [ ]:
plot_prediction(df_arima, df_train)

## MSE

In [ ]:
mse_arima = mean_squared_error(df_test['Precio'].loc[list(df_test.index)], df_arima['Precio'].loc[list(df_test.index)])
mse_arima

# SEASONAL AUTOREGRESSIVE INTEGRATED MOVING AVERAGE MODEL (SARIMA)

Tendremos 3 parámetros nuevos frente al modelo ARIMA, y un parámetro m representando la frecuencia de la estacionalidad de la serie.
En nuestro caso, no se ve gráficamente unos patrones de estacionalidad, por lo cual, es interesante volver al dibujo de decomposición que hicimos al principio. No se ve claramente un patrón, pero sí es cierto que los dos últimos años parece haber patrones. Por motivos prácticos, asumiremos una frecuencia de 12 representando los 12 meses de un año de periodicidad.

In [ ]:
def create_opt_SARIMA(df_to_predict, grades, d, f):
    '''
    Función para fittear distintos SARIMA con todas las combinaciones de grades que le indicaremos, eligiendo el modelo más óptimo (viendo su AIC)
    Si queremos crear un ARMA, habrá que poner la 'd' a cero.
    
    df_to_predict: pandas dataframe
        Es el df ya tratado previamente para poder aplicarle el modelo ARMA y predecir
    
    grades: int MAYOR QUE CERO
        Creará una lista de 0 hasta grades
    
    d: orden de diferenciacion, cuantas veces hay que diferenciar la serie para hacerla estacional
    
    f: frecuencia de la periodicidad
        
    '''
    grades = list(range(0,grades + 1))
    D = 0 
    #https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAXResults.html
    arma_mods = [SARIMAX(df_to_predict, order=(p, d, q), seasonal_order = (P, D, Q, f)).fit(disp = False) for q in grades for p in grades for P in grades for Q in grades]
    aic = [arma.aic for arma in arma_mods]
    ar = [list(arma.model_orders.values())[2]for arma in arma_mods] # Elegimos el valor 2, ya que en model_orders, es donde se encuentra el parametro p correspondiente al ar
    ma = [list(arma.model_orders.values())[3]for arma in arma_mods]
    s_ar = [list(arma.model_orders.values())[4]for arma in arma_mods] 
    s_ma = [list(arma.model_orders.values())[5]for arma in arma_mods] 
    
    df_armas = pd.DataFrame(data = zip(ar, ma, s_ar, s_ma, aic), columns = ["p","q", "P",  "Q", "AIC"])
    df_armas = df_armas.sort_values("AIC") # Ordenamos de MENOR a MAYOR, ya que el menor AIC  
    return df_armas, arma_mods

In [ ]:
import pickle
import os

In [ ]:
os.getcwd()

In [ ]:
update = False # Si el modelo ya ha sido creado, hacemos load poniendo update = False, de lo contrarios, construimos el modelo de 0 poniendo update = True
if update == True:
    df_sarimas, sarima_mods = create_opt_SARIMA(df_diff, 4, d = 1, f = 12)
    
    # Seleccionamos el primer modelo al ser el que tiene menor AIC, por eso elegimos el [0]
    sarima_model = sarima_mods[df_sarimas.index.tolist()[0]]
    sarima_model.summary()
    sarima_model.save('sarima_v2.pkl')
else: 
    sarima_model = pickle.load(open('sarima_v1.pkl', 'rb'))
    print(sarima_model.summary())

In [ ]:
sarima_model.plot_diagnostics(figsize=(10, 8));

In [ ]:
sarima_predict = sarima_model.get_prediction(start = time_to_predict, end = time_test)
prediction_sarima = pd.DataFrame( columns=['predicted_SARIMA'])

In [ ]:
df_pred = pd.DataFrame({'Fecha':sarima_predict.predicted_mean.index, 'Precio':sarima_predict.predicted_mean.values})
df_sarima = pd.concat([df_train, df_pred])
df_sarima = df_sarima.reset_index(drop=True)

df_sarima['Precio'].loc[(len(df_train)-1):,] = df_sarima['Precio'][(len(df_train)-1):].cumsum()
df_sarima


## PLOT SARIMA PREDICTION

In [ ]:
plot_prediction(df_sarima, df_train)

## MSE

In [ ]:
mse_sarima = mean_squared_error(df_test['Precio'].loc[list(df_test.index)], df_sarima['Precio'].loc[list(df_test.index)])
mse_sarima

## MSE Competition

In [ ]:
def plot_prediction(df_total, df):
    fig = make_subplots(rows=1, cols=1)


    train =df_total.loc[:(len(df)-1)]
    prediction = df_total.loc[(len(df)-1):]
    prediction

    fig.add_trace(
        go.Line(name = "Precio Real", x = train.Fecha, y = train.Precio),
        row=
        1, col=1
    )
    fig.add_trace(        
        go.Line(name = "Precio Predicho", x = prediction.Fecha, y = prediction.Precio),
        row=
        1, col=1
    )
    fig
    fig.show()

In [ ]:
df_mse = pd.DataFrame(columns = ['Model', 'MSE'])
df_mse.loc[0]= ['MA', mse_ma]
df_mse.loc[1]= ['AR', mse_ar]
df_mse.loc[2]= ['ARMA', mse_arma]
df_mse.loc[3]= ['ARIMA', mse_arima]
df_mse.loc[4]= ['SARIMA', mse_sarima]
df_mse

In [ ]:
fig_mse = px.bar(df_mse, x='Model', y='MSE', title= 'Competeción MSE de los modelos construidos')
fig_mse.show()
